In [1]:
# %pip install sentence_transformers
# %pip install fastapi
# %pip install uvicorn
# %pip install scikit-learn
# %pip install joblib

# import sentence_transformers
# from sentence_transformers import SentenceTransformer

In [2]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputClassifier
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report
import numpy as np
import joblib
import os

# 1. Cargar el dataset
df = pd.read_csv('../data/processed/books.csv')

# 2. Combinar título y blurb como entrada textual
df['text'] = df['book_title'].fillna('') + '. ' + df['blurb'].fillna('')

# 3. Procesar los tags: convertirlos en listas
df['tags'] = df['tags'].apply(lambda x: [t.strip() for t in str(x).split(',')])

# 4. Binarización multilabel
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['tags'])

# 5. Generar embeddings con un modelo multilingüe
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
X_embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)

# 6. Entrenar modelo multilabel
clf = MultiOutputClassifier(LogisticRegression(max_iter=1000))
clf.fit(X_embeddings, Y)

# 7. Guardar modelo y objetos necesarios
joblib.dump(clf, '../model/book_tagging_pipeline.joblib')
joblib.dump(mlb, '../model/book_tagging_pipeline_mlb.joblib')
joblib.dump(model, '../model/book_tagging_pipeline_sentence_bert.joblib')

# 8. Función para predecir tags de nuevos libros
def predict_tags(titles, blurbs, threshold=0.3):
    texts = [t + '. ' + b for t, b in zip(titles, blurbs)]
    X_test = model.encode(texts)
    
    # Obtener las probabilidades de predicción
    preds_proba = np.array([proba[:, 1] for proba in clf.predict_proba(X_test)]).T
    
    # Aplicar el umbral para convertir probabilidades en etiquetas binarias
    preds = (preds_proba >= threshold).astype(int)
    
    # Invertir la binarización para obtener las etiquetas originales
    tag_lists = mlb.inverse_transform(preds)
    return tag_lists

# Ejemplo de predicción
new_titles = ["La conspiración del universo"]
new_blurbs = ["Una historia que entrelaza ciencia, fe y filosofía para revelar los secretos de la creación."]
predicted_tags = predict_tags(new_titles, new_blurbs)
print(predicted_tags)

Batches:   0%|          | 0/37 [00:00<?, ?it/s]

[('astronomy', 'philosophy', 'religion', 'science', 'to-read')]


In [3]:
# Ejemplo generación de embeddings para la "Isla del Tesoro" en inglés
new_titles = ["Treasure Island"]
new_blurbs = ["Treasure Island is an adventure novel by Scottish author Robert Louis Stevenson, narrating a tale of buccaneers and buried gold."]
predicted_tags = predict_tags(new_titles, new_blurbs)
print(predicted_tags)

[('catholic', 'classics', 'fiction', 'literary-criticism', 'literature', 'to-read')]


In [4]:
# Ejemplo generación de embeddings para la "Isla del Tesoro" en español
new_titles = ["La isla del tesoro"]
new_blurbs = ["La isla del tesoro es una novela de aventuras del autor escocés Robert Louis Stevenson, que narra una historia de bucaneros y oro enterrado."]
predicted_tags = predict_tags(new_titles, new_blurbs)
print(predicted_tags)

[('catholic', 'classics', 'fiction', 'literary-criticism', 'literature', 'to-read')]


In [5]:
# Ejemplo generación de embeddings para "El Quijote"
new_titles = ["Don Quijote de la Mancha"]
new_blurbs = ["Don Quijote de la Mancha es una novela escrita por el español Miguel de Cervantes Saavedra. Publicada su primera parte con el título de El ingenioso hidalgo don Quijote de la Mancha a comienzos de 1605, es una de las obras más destacadas de la literatura española y la literatura universal."]
predicted_tags = predict_tags(new_titles, new_blurbs)
print(predicted_tags)

[('catholic', 'classics', 'literature', 'spain', 'to-read')]
